### Cell 0 – Repository Setup (Colab Only)

**Purpose**: This cell exists only for Google Colab execution. It ensures the correct repository and branch are downloaded before running the experiment.

**What Happens**:
- Clones the GitHub repository
- Selects the trajectory_tracking_policy branch
- Sets the project root so Python can resolve imports correctly

In [ ]:
import sys
import os

# 1. Clone the repository
!git clone -b trajectory_tracking_policy https://github.com/orc-podavini-grisenti/final_assignment

# 2. Point the project_root specifically to the cloned folder
# In Colab, the folder structure will be /content/final_assignment/
project_root = os.path.abspath('/content/final_assignment')

# 3. Add that specific folder to the system path
if project_root not in sys.path:
    sys.path.append(project_root)


# 4. Link the Drive to the colab in order to store the results
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive/final_assignment/outputs/models_saved"

# Crea la cartella se non esiste
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

fatal: destination path 'final_assignment' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Cell 1: Imports & Global Setup
This cell initializes all core dependencies and defines shared configuration variables.

In [1]:
import sys
import os
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

# --- PATH SETUP ---
# Add the project root to sys.path so we can import envs/planner
# Adjust '..' depending on where this notebook is located relative to root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from envs.unicycle_env import UnicycleEnv
from planner.dubins_planner import DubinsPlanner
from models.trajectory_tracking_network import TTNetwork
from models.value_network import ValueNetwork
from utils.normalization import ObservationNormalizer
from utils.reward import TrajectoryTrackingReward

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

Running on: cuda


### Cell 2: Experiment Configuration 
Defines hyperparameters and experiment-level settings; plus intialize the directory for all the output of the training

In [25]:
# Hyperparameters
MAX_EPISODES = 4000
MAX_STEPS    = 200
POLICY_LR     = 1e-3
BATCH_SIZE   = 32
GAMMA        = 0.99

SEED         = 0        # Seed for reproducibility
PATIENCE     = 10       # Stop after 10 updates (200 episodes) without improvement

EXPERIMENTS_PATH = drive_path if 'drive_path' in locals() else "experiments/"


# Global Output Directory
run_id = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
output_dir = os.path.join(EXPERIMENTS_PATH, run_id)
os.makedirs(output_dir, exist_ok=True)
print(f"Results will be saved to: {output_dir}")

Results will be saved to: experiments/run_20260109_100247


### Cell 3 – Environment Initialization
Creates the reinforcement learning environment.

In [ ]:
# MENAGE THE SEED for REPRODUCIBILITY 
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"Seed set to: {SEED}")


# INIT THE ENVIROMENT 
env = UnicycleEnv('empty_env')

# INIT THE REWARD CALCULATOR  
reward_calculator = TrajectoryTrackingReward()


# HELPER
def sample_radius(mean=1.65, std=0.3, min_r=1.0, max_r=2.5):
    """Samples a radius from a truncated Gaussian distribution."""
    radius = np.random.normal(loc=mean, scale=std)
    return np.clip(radius, min_r, max_r)

def sample_seed(min_eval_seed=100):
    """Samples a seed, skipping the first 100 reserved for evaluation."""
    return random.randint(min_eval_seed, 1000000)

Seed set to: 0


### 4. Networks Initialization

**TTNetwork (Trajectory Tracking Network):** This is our **Policy Network (Actor)**. It learns the mapping from states to actions, determining the best step to take at any given time. It is a simple multi layer netweork. With the network we have also:

* Adam Optimizer + a scheduler to reduce the learning rate
* Normalizer of the Observable, which normalize the input of the network

In [27]:
# POLICY NETWORK
OBS_DIM = 3     # Obs dim is 3 (rho, alpha, d_theta)
ACTION_DIM = 2  # v, omega

policy = TTNetwork(OBS_DIM, ACTION_DIM).to(device)
optimizer = optim.Adam(policy.parameters(), lr=POLICY_LR)
# This helps the network "settle" into the precise solution
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5) # Reduces LR by factor of 0.5 every 200 episodes

# Normalizer for the Policy Input
obs_normalizer = ObservationNormalizer(max_dist=3.0)

### Cell 5: Data Managmement

The TrajectoryBuffer class acts as the central repository for all agent experiences accumulated during the training. In this way we have a clean and structured data management

**Key Logic**: Returns-to-GoThe finish_episode method implements the discounted return calculation ($G_t$). This converts immediate rewards into a long-term signal, which is essential for the Policy Gradient update:
$$G_t = \sum_{k=0}^{\infty} \gamma^k r_{t+k}$$

In [28]:
class TrajectoryBuffer:
    def __init__(self):
        # --- 1. Current Episode Storage ---
        self.episode_log_probs = [] 
        self.episode_rewards = []
        
        # --- 2. Batch Storage (Accumulated for the Update) ---
        self.batch_log_probs = []   
        self.batch_returns = []            

        # --- 3. Global History Storage (For Plotting) ---
        self.episode_reward_history = []    # Total reward per episode
        self.batch_reward_history = []      # Avg reward per batch
        self.policy_loss_history = []       # Policy loss per batch update

    def store_step(self, log_prob):
        self.episode_log_probs.append(log_prob)

    def store_reward(self, reward):
        self.episode_rewards.append(reward)

    def store_metrics(self, episode_reward=None, batch_reward=None, policy_loss=None):
        """
        Logs metrics to history. 
        - episode_reward is usually logged every episode.
        - batch_reward, policy_loss and value_loss are usually logged every batch update.
        """
        if episode_reward is not None:
            self.episode_reward_history.append(episode_reward)
        
        if batch_reward is not None:
            self.batch_reward_history.append(batch_reward)

        if policy_loss is not None:
            self.policy_loss_history.append(policy_loss)
            

    def finish_episode(self, gamma):
        """Calculates Returns-to-Go and moves data to batch storage."""
        G_t = 0
        returns = []
        for r in reversed(self.episode_rewards):
            G_t = r + (gamma * G_t)
            returns.insert(0, G_t)
        
        self.batch_log_probs.extend(self.episode_log_probs)
        self.batch_returns.extend(returns)
        
        # Clear episode memory
        self.episode_log_probs = []
        self.episode_rewards = []

    def clear_batch(self):
        self.batch_log_probs = []
        self.batch_returns = []

# Usage
buffer = TrajectoryBuffer()

### 6. Helper Functions

📍 Path Tracking & State Preparation
The primary challenge is transforming a "reach-the-goal" environment into a "follow-the-path" task.

* **`get_tracking_obs`**
    The environment's default observation provides data relative to a final static goal. This function recalculates the state so the agent observes its position relative to the **next waypoint** identified above:
    * **$\rho$**: Distance to the current waypoint.
    * **$\alpha$**: Angle to the waypoint relative to the agent's heading.
    * **$\Delta\theta$**: Orientation error relative to the desired path direction.

* **`get_path_tracking_info`**
    Finding the next waypoint is non-trivial because the agent—especially during early training—may overshoot or miss a point. A simple incremental index is not robust enough. 
    * **Logic:** We identify the waypoint **closest** to the agent's current position and then target the **waypoint immediately following it**. 


🎮 Action Selection

* **`get_action`**
    This function interfaces with the **TTNetwork** to determine movement.Rather than choosing the single "best" action, we sample from the policy's probability distribution. This randomness is crucial during training; it ensures the agent explores different trajectories to find the most efficient path rather than getting stuck in a local optimum.

In [29]:
def normalize_angle(angle):
    """Wraps an angle to the range [-pi, pi]."""
    return (angle + np.pi) % (2 * np.pi) - np.pi

   
def get_tracking_obs(target_waypoint, robot_state):
    """
    Computes rho (distance), alpha (angle error), and d_theta (heading error).
    
    Args:
        target_waypoint: [x, y, theta] of target
        robot_state: [x, y, theta] of current robot position
    """
    # 1. Compute position errors
    dx = target_waypoint[0] - robot_state[0]
    dy = target_waypoint[1] - robot_state[1]
    
    # Rho: Euclidean distance
    rho = np.sqrt(dx**2 + dy**2)
    
    # Alpha: Angle to waypoint relative to robot heading
    # Formula: atan2(dy, dx) - current_theta
    alpha = normalize_angle(np.arctan2(dy, dx) - robot_state[2])
    
    # d_theta: Heading alignment error
    d_theta = normalize_angle(target_waypoint[2] - robot_state[2])

    return np.array([rho, alpha, d_theta], dtype=np.float32)


def get_path_tracking_info(path, robot_state, current_path_idx):
    """
    Calculates tracking error and determines progress along the path.
    
    Returns:
        error (float): Distance to the closest point on the path.
        next_idx (int): The updated waypoint index the robot should target.
        checkpoints_cleared (int): Number of new waypoints passed in this step.
    """
    # 1. Extract (x, y) from the path
    path_xy = path[:, :2]
    
    # 2. Compute distances to all waypoints to find the closest one
    dists = np.linalg.norm(path_xy - robot_state[:2], axis=1)
    error = np.min(dists)
    closest_idx = np.argmin(dists)
    
    # 3. Lookahead Logic:
    # We target the waypoint immediately following the closest one.
    # We use max() to ensure the robot never "goes back" in its target index.
    next_idx = max(current_path_idx, closest_idx + 1)
    next_idx = min(next_idx, len(path) - 1)
    
    # 4. Progress Calculation:
    # This is the "Checkpoints Cleared" logic. 
    # It counts how many waypoints we advanced beyond the previous target.
    checkpoints_cleared = next_idx - current_path_idx
    checkpoints_cleared = max(0, checkpoints_cleared)
    
    return error, next_idx, checkpoints_cleared


def get_action(trajectory_obs):
    """
    Processes observation, samples action, and records transition data.
    """
    # 1. Pre-process and move to device
    # Use the imported normalizer to normaliza the inputs features to give the same relevance
    norm_obs = obs_normalizer.normalize_tt(trajectory_obs)
    obs_t = torch.as_tensor(norm_obs, dtype=torch.float32, device=device).unsqueeze(0)

    # 2. Policy Forward Pass
    mean, std = policy(obs_t)
    dist = Normal(mean, std)
    
    # 3. Sample Action (stochastic for exploration)
    action = dist.sample()
    log_prob = dist.log_prob(action).sum(dim=1)

    # 4. Store transition data in the buffer 
    buffer.store_step(log_prob)

    # 5. Convert to numpy and clip for the environment
    action_np = action.cpu().detach().numpy()[0]
    return np.clip(action_np, -1.0, 1.0)

### 7. REINFORCE Update
This cell implements the training step for the REINFORCE algorithm.

🔄 The Update Process

1.  **Prepare Batch Data:** Convert cumulative returns ($G_t$) and observations into tensors, ensuring shapes are compatible for vectorized processing.
2.  **Calculate Policy Loss:** * Compute the loss using the formula: $L = -\log(\pi(a_t | s_t)) \cdot G_t$.
    * Positive advantage increases action probability; negative advantage decreases it.
3.  **Update Policy Network:** * Apply gradient clipping to prevent exploding gradients and update weights via backpropagation.
4.  **Memory Management:** Clear batch buffers (log_probs, returns, observations) to prevent memory leaks and ensure the next batch starts fresh.

In [30]:
def update():
   
    # 0. Check if we have enough data
    if len(buffer.batch_returns) == 0:
        return

    # 1. PREPARE BATCH DATA:
    # Use torch.stack on the existing log_prob tensors to maintain the grad_fn
    log_probs_t = torch.stack(buffer.batch_log_probs).view(-1, 1)
    # Returns (G_t) are constants, so these don't need gradients
    returns_t = torch.tensor(buffer.batch_returns).float().to(device).view(-1, 1)
    
    # Apply Batch-Wise Normalization to G_t to stabilize the scale of policy updates.
    if len(returns_t) > 1:
        returns_t = (returns_t - returns_t.mean()) / (returns_t.std() + 1e-8)

       
    # 2. CALCULATE POLICY LOSS:
    # Iterate through sampled actions to calculate: 
    #   Policy Loss = 1/N sum_N ( -log(π(u_t | x_t)) * A_t )
    policy_loss = -(log_probs_t * returns_t).mean()
    p_loss_val = policy_loss.item()


    # 3. UPDATE POLICY NETWORK
    # Perform backpropagation to calculate policy gradients.
    optimizer.zero_grad()
    policy_loss.backward()
    # Apply gradient clipping and update policy parameters.
    torch.nn.utils.clip_grad_norm_(policy.parameters(), 1.0)
    optimizer.step()


    # 4. CLEANUP & MEMORY MANAGEMENT:
    # Store metrics and wipe the batch buffer for the next training iteration
    buffer.store_metrics(policy_loss=p_loss_val)
    buffer.clear_batch()


    return p_loss_val

### Cell 8: Training

This cell executes the main training cycle. The process is divided into environment setup, agent interaction, and batch-wise policy updates.

🏋️ The Training Workflow

1.  **Episode Setup:** For each episode, we sample a new random seed and plan a smooth trajectory using a **Dubins Path Planner** with varying curvatures.
2.  **Interaction Loop (The Episode):**
    * **State Transformation:** The raw environment state is converted into an **Ego-Centric Observation** $(\rho, \alpha, \delta\theta)$ relative to the next waypoint.
    * **Action & Environment Step:** The agent samples an action from the policy, moves, and receives a reward based on tracking accuracy and terminal status.
    * **Terminal Conditions:** Episodes end if the agent reaches the goal, collides, wanders too far off-path (`error > 2.0`), or times out.
3.  **Batch Learning:** Every `BATCH_SIZE` episodes, the `update()` function is called to perform backpropagation on both the Policy and Value networks.
4.  **Early Stopping & Checkpointing:**
    * The model monitors the **Average Reward** of the last batch.
    * If a new best performance is achieved, the model weights are saved to disk.
    * If performance plateaus for a defined period (`PATIENCE`), training terminates early to prevent overfitting.

In [31]:
# Training Helpers
best_avg_reward = -float('inf')     # best avarage reward 
patience_counter = 0                # counter of the no improve updates
output_model_path = output_dir + '/policy_model.ph'

print("Starting Training...")

# Run all the episodes
for episode in range(MAX_EPISODES):
    # 1. SETUP EPISODE
    # Each episode is a different enviroment configuration
    seed = sample_seed()
    obs, info = env.reset(seed)


    # Plan path using a sampled radius
    radius = sample_radius()
    planner = DubinsPlanner(curvature=radius, step_size=0.2)
    path = planner.get_path(env.state, env.goal)

    if path is None: 
        continue

    ep_reward, path_idx = 0, 0
    episode_errors = []


    # 2. INTERACTION LOOP (THE EPISODE)
    for t in range(MAX_STEPS):
        # A. Get observation relative to current target waypoint
        # Input of the network is the Ego-Centric Observation Vector: 
        #   1. rho (ρ): [0, inf]        Euclidean distance to the goal.        
        #   2. alpha (α): [-pi, pi]     The angle of the goal *relative* to the robot's current heading.
        #   3. d_theta (δθ): [-pi, pi]  The difference between the desired goal orientation and current heading.
        target_waypoint = path[path_idx]        # The goal is to reach the next waypoint of the path
        tracking_obs = get_tracking_obs(target_waypoint, env.state)       
        
        # B. Agent chooses action and records log_prob/obs in memory
        action = get_action(tracking_obs)

        # C. Environment Step
        _, terminated, truncated, info = env.step(action)

        # D. Update Path Tracking (Find closest waypoint and move target forward)
        error, path_idx, checkpoints_cleared = get_path_tracking_info(path, env.state[:2], path_idx)
        episode_errors.append(error)
        
        # E. Determine Terminal State and Reason
        terminal_reason = None
        if info['is_success']: terminal_reason = "goal"
        elif info['collision']: terminal_reason = "collision"
        elif error > 2.0:       terminal_reason = "off_path"
        elif t >= MAX_STEPS - 1: terminal_reason = "timeout"
        
        # F. Reward Calculation
        # Pass the tracking observation (rho, alpha, d_theta), action, and status
        step_reward = reward_calculator.compute_reward(
            tracking_obs=tracking_obs,
            action=action,
            checkpoints_cleared=checkpoints_cleared,
            terminal_reason=terminal_reason
        )

        buffer.store_reward(step_reward)
        ep_reward += step_reward

        if terminal_reason:
            break


    # 3. EPISODE END: Process Return-to-Go
    buffer.finish_episode(GAMMA)
    buffer.store_metrics(episode_reward=ep_reward)


    # 4. BATCH UPDATE: Perform Learning
    if (episode + 1) % BATCH_SIZE == 0:
        p_loss = update() 
        print(f"Update performed at episode {episode}")

        # Calculate performance over the last batch
        avg_r = np.mean(buffer.episode_reward_history[-BATCH_SIZE:]) # Average reward of the last batch
        buffer.store_metrics(batch_reward=avg_r)
        
        
        # 5. EARLY STOPPING & SAVING
        if avg_r > best_avg_reward:
            best_avg_reward = avg_r
            patience_counter = 0
            # Store the best version of the weights 
            torch.save(policy.state_dict(), output_model_path)
            print(f"Update {episode//BATCH_SIZE} | New Best: {best_avg_reward:.2f} | Model Saved.")
        else:
            patience_counter += 1
            print(f"Update {episode//BATCH_SIZE} | Avg Reward: {avg_r:.2f} | Patience: {patience_counter}/{PATIENCE}")

        if patience_counter >= PATIENCE:
            print(f"Early stopping triggered at episode {episode}")
            break


print(f"Training Complete. Best average reward: {best_avg_reward:.2f}")

Starting Training...
Update performed at episode 31
Update 0 | New Best: 22.17 | Model Saved.
Update performed at episode 63
Update 1 | New Best: 63.46 | Model Saved.
Update performed at episode 95
Update 2 | Avg Reward: 27.72 | Patience: 1/10
Update performed at episode 127
Update 3 | Avg Reward: 50.77 | Patience: 2/10
Update performed at episode 159
Update 4 | New Best: 73.57 | Model Saved.
Update performed at episode 191
Update 5 | New Best: 140.06 | Model Saved.
Update performed at episode 223
Update 6 | Avg Reward: 69.86 | Patience: 1/10
Update performed at episode 255
Update 7 | Avg Reward: 128.30 | Patience: 2/10
Update performed at episode 287
Update 8 | New Best: 158.82 | Model Saved.
Update performed at episode 319
Update 9 | New Best: 192.74 | Model Saved.
Update performed at episode 351
Update 10 | New Best: 196.95 | Model Saved.
Update performed at episode 383
Update 11 | New Best: 233.16 | Model Saved.
Update performed at episode 415
Update 12 | Avg Reward: 229.80 | Patie

### Cell 9: Visualization & Save
Plot the learning curve to verify convergence.

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

def ensure_numpy(data):
    """Helper to convert lists of floats or tensors to a clean numpy array."""
    return np.array([d.detach().cpu().item() if torch.is_tensor(d) else d for d in data])

# 1. Prepare Data
ep_rewards = ensure_numpy(buffer.episode_reward_history)
batch_rewards = ensure_numpy(buffer.batch_reward_history)
p_losses = ensure_numpy(buffer.policy_loss_history)


# 2. Setup Figure
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
plt.suptitle(f"REINFORCE Training Metrics", fontsize=16)

# Plot 1: Total Reward per Episode
axes[0].plot(ep_rewards, color='blue', alpha=0.3, label='Episode Reward')
if len(batch_rewards) > 0:
    x_batch = (np.arange(len(batch_rewards)) + 1) * BATCH_SIZE - 1
    axes[0].plot(x_batch, batch_rewards, color='green', marker='o', 
                    markersize=4, linewidth=2, label='Batch Average')
axes[0].set_title("Reward Progress")
axes[0].set_xlabel("Episode")
axes[0].legend()
axes[0].grid(True)

# Plot 2: Policy Loss
axes[1].plot(p_losses, color='red')
axes[1].set_title("Policy Loss")
axes[1].set_xlabel("Update Batch")
axes[1].grid(True)

# 3. Save and Close
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plot_path = os.path.join(output_dir, "training_metrics.png")
plt.savefig(plot_path)
plt.close() # Prevents duplicate display in notebooks


# 4. SAVING DATA 
# Saves history arrays to disk for future analysis.
data_to_save = {
    "episode_rewards": ep_rewards,
    "policy_losses": p_losses,
}

for name, arr in data_to_save.items():
    np.save(os.path.join(output_dir, f"{name}.npy"), arr)

print("-" * 30)
print(f"Execution complete.")
print(f"Directory: {output_dir}")
print("-" * 30)

------------------------------
Execution complete.
Directory: experiments/run_20260109_100247
------------------------------


### Cell 10: Evaluation

In [ ]:
from evaluation.controllers_evaluation import evaluate_single_model

report_path = output_dir + '/evaluation_report.txt'
evaluate_single_model(output_model_path, "last_run", 50, 0, 
                      verbose=True, save_csv=False, txt_file_path=report_path)

--> Loading Model: v1_no_baseline/policy_model.ph
RL Controller loaded successfully on cuda
Starting 50 episodes...
Progress: 50/50
Evaluation Complete.

 EVALUATION REPORT: last_run
╒══════════════╤══════════════╤═══════════════╤═══════════════════╤══════════════╤═════════════════╕
│ Metric       │ Value        │ Metric        │ Value             │ Metric       │ Value           │
╞══════════════╪══════════════╪═══════════════╪═══════════════════╪══════════════╪═════════════════╡
│ Status       │ ✅ EXCELLENT │ Mean CTE      │ 0.0103 ± 0.0015 m │ Avg Velocity │ 0.95 ± 0.01 m/s │
├──────────────┼──────────────┼───────────────┼───────────────────┼──────────────┼─────────────────┤
│ Success Rate │ 100.0 %      │ Max CTE (Avg) │ 0.0282 m          │ Smoothness   │ 0.0559 ± 0.0092 │
├──────────────┼──────────────┼───────────────┼───────────────────┼──────────────┼─────────────────┤
│ Steps (Avg)  │ 295.1 ± 26.8 │ Tortuosity    │ 1.000 ± 0.001     │ Avg Energy   │ 15.15 ± 1.34    │
╘═════════

### Cell 11: ⚠ Delete the train directory ( only for bed results )

In [ ]:
import shutil

# --- DELETE CURRENT EXPERIMENT DIRECTORY ---
# Ask for confirmation to prevent accidental deletion
confirm = input(f"Do you want to delete the directory '{run_id}'? (y/n): ")

if confirm.lower() == 'y':
    try:
        # shutil.rmtree deletes a directory and all its contents
        shutil.rmtree(output_dir)
        print(f"Successfully deleted: {output_dir}")
    except OSError as e:
        print(f"Error: {e.strerror}. Could not delete the directory.")
else:
    print("Deletion cancelled. Experiment results preserved.")